<a href="https://colab.research.google.com/github/la2015-hw/Group_10/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaishnavichintha","key":"7f7453401b6964fd1e8a85d1dc7c0977"}'}

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
!kaggle datasets list

ref                                                            title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset   Global Earthquake-Tsunami Risk Assessment Dataset        16151  2025-10-01 16:35:53.273000          13997        476  1.0              
jockeroika/life-style-data                                     Life Style Data                                        3995645  2025-10-14 13:50:45.303000          16402        341  0.8235294        
jaderz/hospital-beds-management                                Hospital Beds Management                                 47583  2025-10-03 09:21:58.590000          11443        293  1.0              
alams

In [9]:
!kaggle datasets download -d vaishnavichintha/sdss-tabular-dataset

Dataset URL: https://www.kaggle.com/datasets/vaishnavichintha/sdss-tabular-dataset
License(s): unknown
  0% 0.00/10.7M [00:00<?, ?B/s]
100% 10.7M/10.7M [00:00<00:00, 1.65GB/s]


In [10]:
!unzip sdss-tabular-dataset.zip

Archive:  sdss-tabular-dataset.zip
  inflating: sdss_gz2_matched_cleaned_balanced.csv  


In [11]:
import pandas as pd
df = pd.read_csv("sdss_gz2_matched_cleaned_balanced.csv")
print(df.head())


             specobjid      dr8objid            dr7objid          ra  \
0   614788390251948032  1.237654e+18  587728665578700971  122.952644   
1  2433194008562919424  1.237665e+18  587739651567845579  231.428600   
2  2080728518915811328  1.237663e+18  587736975809773641  232.551060   
3  1563980851195176960  1.237662e+18  587736619321917717  234.050800   
4  3139079110530197504  1.237666e+18  587739843772940458  217.789000   

         dec     rastring    decstring    sample gz2class  \
0  42.216805  08:11:48.63  +42:13:00.5  original    SBc?m   
1  23.370867  15:25:42.86  +23:22:15.1  original      Ser   
2  27.134108  15:30:12.25  +27:08:02.8  original    SBc2m   
3  30.967339  15:36:12.19  +30:58:02.4  original    SBd2m   
4  20.449600  14:31:09.36  +20:26:58.6  original    SBa2l   

   total_classifications  ...  \
0                     47  ...   
1                     40  ...   
2                     44  ...   
3                     55  ...   
4                     34  ...   

 

In [12]:
df.info()
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47600 entries, 0 to 47599
Columns: 234 entries, specobjid to broad_class
dtypes: float64(151), int64(78), object(5)
memory usage: 85.0+ MB
Index(['specobjid', 'dr8objid', 'dr7objid', 'ra', 'dec', 'rastring',
       'decstring', 'sample', 'gz2class', 'total_classifications',
       ...
       't11_arms_number_a36_more_than_4_weighted_fraction',
       't11_arms_number_a36_more_than_4_debiased',
       't11_arms_number_a36_more_than_4_flag',
       't11_arms_number_a37_cant_tell_count',
       't11_arms_number_a37_cant_tell_weight',
       't11_arms_number_a37_cant_tell_fraction',
       't11_arms_number_a37_cant_tell_weighted_fraction',
       't11_arms_number_a37_cant_tell_debiased',
       't11_arms_number_a37_cant_tell_flag', 'broad_class'],
      dtype='object', length=234)


In [17]:
# Separate target first
y = df["broad_class"]

# Drop irrelevant columns (IDs, coordinates, etc.)
drop_cols = [
    "specobjid", "dr8objid", "dr7objid",
    "ra", "dec", "rastring", "decstring",
    "sample", "gz2class", "broad_class"  # drop target from features
]

x = df.drop(columns=drop_cols, errors="ignore")

print("Feature shape:", x.shape)
print("Target shape:", y.shape)


Feature shape: (47600, 224)
Target shape: (47600,)


In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

print("Scaled feature matrix shape:", x_scaled.shape)


Scaled feature matrix shape: (47600, 224)


In [24]:
from sklearn.model_selection import train_test_split

# 70% train, 30% test
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y, test_size=0.3, random_state=25, stratify=y
)

print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)


Train shape: (33320, 224) (33320,)
Test shape: (14280, 224) (14280,)


In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize KNN model (start with k=3)
knn = KNeighborsClassifier(n_neighbors=5)

# Train (fit) the model
knn.fit(x_train, y_train)

# Predict on test data
y_pred = knn.predict(x_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.4f}")

# More detailed metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Test Accuracy: 0.8325

Classification Report:
                 precision    recall  f1-score   support

     Elliptical       0.79      0.86      0.82      3570
      Irregular       0.89      0.95      0.92      3570
  Spiral-barred       0.86      0.83      0.84      3570
Spiral-unbarred       0.80      0.68      0.74      3570

       accuracy                           0.83     14280
      macro avg       0.83      0.83      0.83     14280
   weighted avg       0.83      0.83      0.83     14280


Confusion Matrix:
[[3071  148   70  281]
 [  81 3407   31   51]
 [ 195  121 2975  279]
 [ 564  171  400 2435]]
